In [ ]:
import re
import json
import pymorphy3
import tokenize_uk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [ ]:
df = pd.read_csv('../../data/prepared_candidates/candidates_prepared.csv')
df.head()

In [ ]:
# check phone number
def detect_phone(text):
    if re.search(r'380\d{9}', text.replace(' ','')) or re.search(r'0\d{9}', text.replace(' ','')):
        return 1
    return 0

def get_phone(text):
    if re.search(r'380\d{9}', text.replace(' ','')):
        return re.search(r'380\d{9}', text.replace(' ','')).group()
    elif re.search(r'0\d{9}', text.replace(' ','')):
        return re.search(r'0\d{9}', text.replace(' ','')).group()
    return None

count = 0
for cv_text in tqdm(df['CV'].values):
    count+=detect_phone(cv_text)
print("Count of CVs with phone number in CVs:", count)
print("Percentage of CVs with phone number in CVs:", 100*count/len(df))

# get phone
df['phone'] = df['CV'].apply(get_phone)
# show 10 random phone
df['phone'].unique()[np.random.randint(0, len(df['phone'].unique()), 10)]

In [ ]:
# check email
def detect_email(text):
    if re.search(r'.+[@ _]gmail\.com', text.replace(' ','')) or re.search(r'.+[@ _]ukr\.net', text.replace(' ','')) or re.search(r'.+[@ _]mail\.ru', text.replace(' ','')):
        return 1
    return 0

def get_email(text):
    if re.search(r'.+[@ _]gmail\.com', text.replace(' ','')):
        return re.search(r'.+[@ _]gmail\.com', text.replace(' ','')).group()
    elif re.search(r'.+[@ _]ukr\.net', text.replace(' ','')):
        return re.search(r'.+[@ _]ukr\.net', text.replace(' ','')).group()
    elif re.search(r'.+[@ _]mail\.ru', text.replace(' ','')):
        return re.search(r'.+[@ _]mail\.ru', text.replace(' ','')).group()
    return None

count = 0
for cv_text in tqdm(df['CV'].values):
    count+=detect_email(cv_text)
print("Count of CVs with email in CVs:", count)
print("Percentage of CVs with email in CVs:", 100*count/len(df))

# get email
df['email'] = df['CV'].apply(get_email)
# show 10 random emails
df['email'].unique()[np.random.randint(0, len(df['email'].unique()), 10)]

In [ ]:
# check вул or street
def detect_street(text):
    if re.search(r'вул\.', text) or re.search(r'home address', text):
        return 1
    return 0

def get_street(text):
    if re.search(r'вул\.', text):
        return re.search(r'вул\..+', text).group()
    elif re.search(r'home address', text):
        return re.search(r'home address.+', text).group()
    return None

count = 0
for cv_text in tqdm(df['CV'].values):
    count+=detect_street(cv_text)
print("Count of CVs with вул. in CVs:", count)
print("Percentage of CVs with вул. in CVs:", 100*count/len(df))

# get street
df['street'] = df['CV'].apply(get_street)
# show 10 random streets
df['street'].unique()[:10]

In [ ]:
# check ІПН, РНОКПП, УНЗР, SNN
def detect_ipn(text):
    if re.search(r'ІПН', text) or re.search(r'РНОКПП', text) or re.search(r'УНЗР', text) or re.search(r'SNN:', text):
        print(text)
        return 1
    return 0

def get_ipn(text):
    if re.search(r'ІПН', text):
        return re.search(r'ІПН.+', text).group()
    elif re.search(r'РНОКПП', text):
        return re.search(r'РНОКПП.+', text).group()
    elif re.search(r'УНЗР', text):
        return re.search(r'УНЗР.+', text).group()
    elif re.search(r'SNN:', text):
        return re.search(r'SNN:.+', text).group()
    return None

count = 0
for cv_text in tqdm(df['CV'].values):
    count+=detect_ipn(cv_text)
print("Count of CVs with ІПН in CVs:", count)
print("Percentage of CVs with ІПН in CVs:", 100*count/len(df))

# get ІПН
df['ipn'] = df['CV'].apply(get_ipn)
# show 10 random ІПН
df['ipn'].unique()[:10] # noy find

In [ ]:
# check social media links
def detect_social(text):
    if re.search(r"w{3}\.linkedin", text) or re.search(r"w{3}\.facebook", text) or re.search(r"w{3}\.instagram", text) or re.search(r"w{3}\.twitter", text) or re.search(r"w{3}\.vk", text) or re.search(r"w{3}\.telegram", text) or re.search(r"w{3}\.viber", text) or re.search(r"w{3}\.whatsapp", text):
        return 1
    return 0

def get_social(text):
    social = []
    if re.search(r"w{3}\.linkedin", text):
        social.append(re.search(r"w{3}\.linkedin.+", text).group())
    if re.search(r"w{3}\.facebook", text):
        social.append(re.search(r"w{3}\.facebook.+", text).group())
    if re.search(r"w{3}\.instagram", text):
        social.append(re.search(r"w{3}\.instagram.+", text).group())
    if re.search(r"w{3}\.twitter", text):
        social.append(re.search(r"w{3}\.twitter.+", text).group())
    if re.search(r"w{3}\.vk", text):
        social.append(re.search(r"w{3}\.vk.+", text).group())
    if re.search(r"w{3}\.telegram", text):
        social.append(re.search(r"w{3}\.telegram.+", text).group())
    if re.search(r"w{3}\.viber", text):
        social.append(re.search(r"w{3}\.viber.+", text).group())
    if re.search(r"w{3}\.whatsapp", text):
        social.append(re.search(r"w{3}\.whatsapp.+", text).group())
    return str(social)

count = 0
for cv_text in tqdm(df['CV'].values):
    count+=detect_social(cv_text)
print("Count of CVs with social media links in CVs:", count)
print("Percentage of CVs with social media links in CVs:", 100*count/len(df))

# get social media links
df['social'] = df['CV'].apply(get_social)
# show 10 random social media links
df['social'].unique()[np.random.randint(0, len(df['social'].unique()), 10)]

In [ ]:
# find socila media nicknames
def find_social_nickname(text):
    text = text.lower()
    if re.search(r"linkedin:", text) or re.search(r"facebook:", text) or re.search(r"instagram:", text) or re.search(r"twitter:", text) or \
        re.search(r"vk:", text) or re.search(r"vkontakte:", text) or re.search(r"telegram:", text) or re.search(r"viber:", text) or re.search(r"whatsapp:", text):
        return 1
    return 0

def get_social_nickname(text):
    social = []
    if re.search(r"linkedin:.+", text):
        social.append(re.search(r"linkedin:.+", text).group())
    if re.search(r"facebook:.+", text):
        social.append(re.search(r"facebook:.+", text).group())
    if re.search(r"instagram:.+", text):
        social.append(re.search(r"instagram:.+", text).group())
    if re.search(r"twitter:.+", text):
        social.append(re.search(r"twitter:.+", text).group())
    if re.search(r"vk:.+", text):
        social.append(re.search(r"vk:.+", text).group())
    if re.search(r"vkontakte:.+", text):
        social.append(re.search(r"vkontakte:.+", text).group())
    if re.search(r"telegram:.+", text):
        social.append(re.search(r"telegram:.+", text).group())
    if re.search(r"viber:.+", text):
        social.append(re.search(r"viber:.+", text).group())
    if re.search(r"whatsapp:.+", text):
        social.append(re.search(r"whatsapp:.+", text).group())
    return str(social)

count = 0
for cv_text in tqdm(df['CV'].values):
    count+=find_social_nickname(cv_text)
print("Count of CVs with social media nicknames in CVs:", count)
print("Percentage of CVs with social media nicknames in CVs:", 100*count/len(df))

# get social media nicknames
df['social_nickname'] = df['CV'].apply(get_social_nickname)
# show 10 random social media nicknames
df['social_nickname'].unique()[np.random.randint(0, len(df['social_nickname'].unique()), 10)]

In [ ]:
df.social = df.social.astype(str)
df = df[(df.social_nickname != '[]') | (df.social != '[]') | (df.ipn.notna()) | (df.street.notna()) | (df.email.notna()) |  (df.phone.notna())]
print(df.shape)
df.head()

In [ ]:
df.to_csv('../data/PII_CV.csv', index=False)

## Delete PII from datasets

In [ ]:
data = pd.read_csv('../../data/djinni/candidates.csv')

In [ ]:
data

In [ ]:
df = df[data.columns]

In [ ]:
#delete all rows from data which we have in df. for deleting use match by all columns
data = data[~data.apply(tuple,1).isin(df.apply(tuple,1))]


In [ ]:
data

In [ ]:
100*(295094-294825)/295094

In [ ]:
data.to_csv('../../data/djinni/candidates.csv', index=False)

In [ ]:
# delete from preprocessed datasets
data = pd.read_csv('../../data/prepared_candidates/candidates_prepared.csv')
print(data.shape)
df = pd.read_csv('../data/PII_CV.csv')
# drop by id
data = data[~data.id.isin(df.id)]
print(data.shape)
data.to_csv('../../data/prepared_candidates/candidates_prepared.csv', index=False)

In [ ]:
# delete from preprocessed datasets
data = pd.read_csv('../../data/prepared_candidates/intermediate_candidates_prepared.csv')
print(data.shape)
df = pd.read_csv('../data/PII_CV.csv')
# drop by id
data = data[~data.id.isin(df.id)]
print(data.shape)
data.to_csv('../../data/prepared_candidates/intermediate_candidates_prepared.csv', index=False)